In [66]:
import tensorflow as tf
import pandas as pd

#df_4forums = pd.read_json('4ForumsResults.json') # This file is too large to go through git
#df_conme = pd.read_excel('convinceMe.xlsx')
df_credb = pd.read_excel('createDebate.xlsx')


In [2]:
# install transformers if not already installed
%pip install transformers

#import the BERT model and tokenizer
from transformers import BertModel, BertTokenizer

if 'bertModel' not in locals() or 'tokenizer' not in locals():
    #BERT base model
    bertModel = BertModel.from_pretrained('bert-base-uncased')
    
    #BERT Tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Note: you may need to restart the kernel to use updated packages.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [67]:
df_credb.stance.value_counts()

unknown                       3051
prefers strict gun control    3051
opposes strict gun control    3051
undecided                     3051
other                         3051
Name: stance, dtype: int64

In [68]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [69]:
class BertClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = model
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits

I'm gonna work with the "Create Debate" Dataset for now to save time

In [108]:
df_credb.stance.value_counts()

prefers strict gun control    3051
opposes strict gun control    3051
Name: stance, dtype: int64

In [109]:
df_credb = df_credb.query("stance == 'prefers strict gun control' | stance == 'opposes strict gun control'")

In [110]:
df_credb = df_credb[["text", "stance"]]

In [111]:
df_credb

,text,stance
642,Guns should be banned because they are not nee...,prefers strict gun control
643,"Yes, guns should be banned. Guns provide a tri...",prefers strict gun control
644,"If you look at history, the first thing empire...",prefers strict gun control
645,Yes that might be true but are you suggesting ...,prefers strict gun control
646,Gun control is misguided. When guns become ill...,prefers strict gun control
...,...,...
15240,"Yes, the fire arms should be banned. But if th...",opposes strict gun control
15241,Herein lies one of my biggest contentions rega...,opposes strict gun control
15242,Making firearms illegal only stops law abiding...,opposes strict gun control
15243,"Yeah, I thought that was hilarious.",opposes strict gun control


In [125]:
#Encoding the stance, which is our target class
import numpy as np

if type(df_credb.stance.iloc[1]) is str:
    class_mapping = {label: idx for idx, label in 
                    enumerate(np.unique(df_credb['stance']))}

class_mapping

{'opposes strict gun control': 0, 'prefers strict gun control': 1}

In [126]:
#Prevents accidentally overwriting the 
if type(df_credb.stance.iloc[1]) is str:
    df_credb['stance'] = df_credb['stance'].map(class_mapping)

df_credb.stance.value_counts()

1    3051
0    3051
Name: stance, dtype: int64

In [127]:
df_credb

,text,stance
642,Guns should be banned because they are not nee...,1
643,"Yes, guns should be banned. Guns provide a tri...",1
644,"If you look at history, the first thing empire...",1
645,Yes that might be true but are you suggesting ...,1
646,Gun control is misguided. When guns become ill...,1
...,...,...
15240,"Yes, the fire arms should be banned. But if th...",0
15241,Herein lies one of my biggest contentions rega...,0
15242,Making firearms illegal only stops law abiding...,0
15243,"Yeah, I thought that was hilarious.",0
